In [ ]:
!pip install -qq boto3
!pip install -qq unidecode

     |████████████████████████████████| 131 kB 5.1 MB/s 
     |████████████████████████████████| 8.0 MB 23.9 MB/s 
     |████████████████████████████████| 79 kB 5.1 MB/s 
     |████████████████████████████████| 138 kB 34.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 235 kB 5.1 MB/s 


In [ ]:
import string
import unidecode
import os
import boto3 
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
access_key = 'aws key'
secret_key =  'aws key'
bucket_name = 'aws key'

Mounted at /content/drive


In [ ]:
def str_lower_rm_punctuation_rm_accents(text):
    """
    """

    if len(text) == 0:
        return ''

    # Puctuations to remove
    table = str.maketrans({
        key: None
        for key in string.punctuation
    })

    # Make sure to have a string
    text_cleant = str(text)

    # Remove Symbols
    text_cleant = text_cleant.translate(table)

    # Remove accents
    text_cleant = unidecode.unidecode(text_cleant)

    # Lower
    text_cleant = text_cleant.lower()

    return str(text_cleant)


In [ ]:
path  = "/content/drive/MyDrive/hackatonBBVA2021/csv/"
path_santander  = path+"Santander/"
path_banorte  = path+"Banorte/"
path_bbva  = path+"BBVA/"
file_banorte = "estado_de_resultados_banorte.csv"
file_bbva = "result_state.csv"
file_santander = "estado_de_resultados_santander.csv"

In [ ]:
dic_santander = {"Utilidad neta mayoritaria": "Resultado neto"}
dic_banorte = {"Ingresos de Intereses Netos antes de Estim .": "Margen Financiero",
               "Ingresos de Intereses Netos Ajustado por": "Margen financiero ajustado por riesgos crediticios",
               "Provisiones Prev. para Riesgos crediticios": "Estimación preventiva para riesgos crediticios",
               "Resultados Antes de Im puestos a la Utilidad":"Resultado antes de impuestos a la utilidad",
               "Resultados antes de operaciones":"Resultado antes de operaciones discontinuadas",
               "Com isiones por Servicios Cobrados": "Comisiones y tarifas cobradas",
               "Com isiones por Servicios Pagados": "Comisiones y tarifas pagados",
               "Ingresos por Interm ediación":"Resultado por intermediación",
               "Total Otros Ingresos (Egresos) de la":"Otros ingresos (egresos) de la operación",
               "Total Gasto No Financiero":"Gastos de administración y promoción",
               "Participación de los Trabajadores en las Utilidades":"Participación en el resultado de subsidiarias y asociadas",
               "Utilidad Neta" : "Resultado neto"
               }

result_state_important_variables = ["resultado neto","margen financiero","margen financiero ajustado por riesgos crediticios",
                                    "resultado de la operacion","resultado antes de impuestos a la utilidad",
                                    "resultado antes de operaciones discontinuadas","resultado neto"]

In [ ]:
# reading files
santander_df = pd.read_csv(path_santander+file_santander)
bbva_df = pd.read_csv(path_bbva+file_bbva)
bbva_df = bbva_df.iloc[: , 1:]
bbva_df.columns=["trimester_date","bank_name","table_name","short_description",
                                   "T_0_feature_value","T_1_feature_value","T_2_feature_value","T_3_feature_value","T_4_feature_value"]
banorte_df = pd.read_csv(path_banorte+file_banorte)
banorte_df.columns=["trimester_date","bank_name","table_name","short_description",
                                   "T_0_feature_value","T_1_feature_value","T_2_feature_value","T_3_feature_value","T_4_feature_value"]
#deleting unnecesary files of bbva
bbva_df = bbva_df.loc[(bbva_df["short_description"]!="Estado de Resultados Consolidado") &
                      (bbva_df["short_description"]!="Millones de pesos")]

#renaming and creating one more file in santander
banorte_df = banorte_df.replace({"short_description": dic_banorte})
santander_df = santander_df.replace({"short_description": dic_santander})
aux = santander_df.loc[santander_df["short_description"]=="Resultado neto"].reset_index(drop = True)
aux["short_description"] = "Resultado antes de operaciones discontinuadas"
santander_df =pd.concat([santander_df,aux])

#deleting unnesary parenthesis
columnas = banorte_df.columns
for columna in columnas:
      banorte_df[columna] = banorte_df[columna].str.replace("(","-") 
      banorte_df[columna] = banorte_df[columna].str.replace(")","")
      banorte_df[columna] = banorte_df[columna].str.replace(",","")
      bbva_df[columna] = bbva_df[columna].str.replace("(","-") 
      bbva_df[columna] = bbva_df[columna].str.replace(")","")
      bbva_df[columna] = bbva_df[columna].str.replace(",","")

#concat
santander_df["table_name"] = "Estado de resultados"
result_state = pd.concat([bbva_df,banorte_df,santander_df])

#to float
result_state["T_0_feature_value"] = result_state["T_0_feature_value"].astype(float)
result_state["T_1_feature_value"] = result_state["T_1_feature_value"].astype(float)
result_state["T_2_feature_value"] = result_state["T_2_feature_value"].astype(float)
result_state["T_3_feature_value"] = result_state["T_3_feature_value"].astype(float)
result_state["T_4_feature_value"] = result_state["T_4_feature_value"].astype(float)

#new vars 
result_state["T_5_feature_value"] = 0
result_state["T_6_feature_value"] = 0

# cleaning short descrption
result_state["short_description"] = result_state["short_description"].apply(lambda x: str_lower_rm_punctuation_rm_accents(str(x)))

In [ ]:
result_state.loc[result_state["short_description"]==result_state_important_variables[1]]

,trimester_date,bank_name,table_name,short_description,T_0_feature_value,T_1_feature_value,T_2_feature_value,T_3_feature_value,T_4_feature_value,T_5_feature_value,T_6_feature_value
7,2021T2,BBVA,Estado de resultados,margen financiero,34409.0,34759.0,35914.0,34045.0,29089.0,0,0
8,2021T2,Banorte,Estado de resultados,margen financiero,19649.0,20002.0,19580.0,20601.0,20683.0,0,0
0,2021T2,Santander,Estado de resultados,margen financiero,15585.0,16272.0,16896.0,15585.0,16896.0,0,0


In [ ]:
path  = "/content/drive/MyDrive/hackatonBBVA2021/csv/"
path_santander  = path+"Santander/"
path_banorte  = path+"Banorte/"
path_bbva  = path+"BBVA/"
file_banorte = "balancegral_pasivos_banorte.csv"
file_bbva = "BBVA_21T2_pasivo.csv"
#file_bbva = "BBVA_21T2_activo.csv"
file_santander = "balance_general_pasivo_santander.csv"
# reading files
santander_df = pd.read_csv(path_santander+file_santander)
santander_df["table_name"] = 'Balance General Pasivos'
santander_df=santander_df[["trimester_date","bank_name","table_name","short_description",
                                   "T_0_feature_value","T_1_feature_value","T_2_feature_value","T_3_feature_value","T_4_feature_value"]]
bbva_df = pd.read_csv(path_bbva+file_bbva)
bbva_df = bbva_df.iloc[: , 1:]
rows,cols = bbva_df.shape
tabla=[{'trimester_date': "2021T2",
 'bank_name': 'BBVA',
 'table_name': 'Balance General Pasivos',
 'short_descripction': bbva_df.iloc[row,0],
 'T_0_feature_value': bbva_df.iloc[row,1] ,
 'T_1_feature_value': bbva_df.iloc[row,2] ,
 'T_2_feature_value': bbva_df.iloc[row,3] ,
 'T_3_feature_value': bbva_df.iloc[row,4],
 'T_4_feature_value': bbva_df.iloc[row,5]} for row in range(rows)] 
bbva_df = pd.DataFrame.from_records(tabla)
bbva_df.columns=["trimester_date","bank_name","table_name","short_description",
                                   "T_0_feature_value","T_1_feature_value","T_2_feature_value","T_3_feature_value","T_4_feature_value"]
banorte_df = pd.read_csv(path_banorte+file_banorte)
banorte_df.columns=["trimester_date","bank_name","table_name","short_description",
                                   "T_0_feature_value","T_1_feature_value","T_2_feature_value","T_3_feature_value","T_4_feature_value"]
#
banorte_df = banorte_df.loc[~((banorte_df["short_description"]=="Derivados") &
                     (banorte_df["T_0_feature_value"]=="0") )
                      ]

#deleting unnecesary files of bbva
bbva_df = bbva_df.loc[(bbva_df["T_0_feature_value"]!="Jun") &
                     (bbva_df["T_0_feature_value"]!="2020") & 
                      (bbva_df["T_0_feature_value"]!="1047863")  
                      ]

dic_santander = {"Prestamos bancarios y de otros organismos": "PRESTAMOS INTERBANCARIOS Y DE OTROS ORGANISMOS",
                 "Total de Colaterales vendidos o dados en":"COLATERALES VENDIDOS O DADOS EN GARANTIA"}
dic_banorte = {"Préstam os de Bancos y  otros":"PRESTAMOS INTERBANCARIOS Y DE OTROS ORGANISMOS",
               "Otras cuentas por  pagar": "Otras cuentas por pagar",
               "Total de Colaterales vendidos o dados en":"COLATERALES VENDIDOS O DADOS EN GARANTIA",
               "TOTAL PASIVOS":"Total pasivo",
               "TOTAL PASIVOS Y CAPITAL":"TOTAL PASIVO Y CAPITAL CONTABLE"}

result_state_important_variables = ["prestamos interbancarios y de otros organismos",
                                    "colaterales vendidos o dados en garantia",
                                    "derivados",
                                    "otras cuentas por pagar",
                                    "total pasivo",
                                    "capital contribuido",
                                    "capital ganado",
                                    "total capital contable",
                                    "total pasivo y capital contable"]
                                    

#renaming and creating one more file in santander
banorte_df = banorte_df.replace({"short_description": dic_banorte})
santander_df = santander_df.replace({"short_description": dic_santander})

#deleting unnesary parenthesis
columnas = banorte_df.columns
for columna in columnas:
      banorte_df[columna] = banorte_df[columna].str.replace("(","-") 
      banorte_df[columna] = banorte_df[columna].str.replace(")","")
      banorte_df[columna] = banorte_df[columna].str.replace(",","")
      bbva_df[columna] = bbva_df[columna].str.replace("(","-") 
      bbva_df[columna] = bbva_df[columna].str.replace(")","")
      bbva_df[columna] = bbva_df[columna].str.replace(",","")

#concat
result_pasive = pd.concat([bbva_df,banorte_df,santander_df])

#to float
result_pasive["T_0_feature_value"] = result_pasive["T_0_feature_value"].astype(float)
result_pasive["T_1_feature_value"] = result_pasive["T_1_feature_value"].astype(float)
result_pasive["T_2_feature_value"] = result_pasive["T_2_feature_value"].astype(float)
result_pasive["T_3_feature_value"] = result_pasive["T_3_feature_value"].astype(float)
result_pasive["T_4_feature_value"] = result_pasive["T_4_feature_value"].astype(float)

#new vars 
result_pasive["T_5_feature_value"] = 0
result_pasive["T_6_feature_value"] = 0

# cleaning short descrption
result_pasive["short_description"] = result_pasive["short_description"].apply(lambda x: str_lower_rm_punctuation_rm_accents(str(x)))

In [ ]:
# folder location
path  = "/content/drive/MyDrive/hackatonBBVA2021/csv/"

path_bbva  = path+"BBVA/"
path_banorte  = path+"Banorte/"
path_santander  = path+"Santander/"

file_bbva = "BBVA_21T2_activo_pivot.csv"
file_banorte = "balancegral_activos_banorte.csv"
file_santander = "balance_general_activo_santander.csv"

bbvadf = path_bbva+file_bbva
banortedf = path_banorte+file_banorte
santanderdf = path_santander+file_santander
# read csvs
df_bbva = pd.read_csv(bbvadf)
df_bbva = df_bbva[2:]
df_bbva.reset_index(drop=True, inplace=True)
df_banorte = pd.read_csv(banortedf)
df_santander = pd.read_csv(santanderdf)

important_variables = ['Inversiones en Valores', 'Total Cartera de Credito Vigente', 'Total Cartera de Credito Vencida',
                       'Total Cartera de Credito', 'Total de Cartera de Credito (Neto)', 'Total Activo']

col_names = ['trimester_date', 'bank_name', 'table_name', 'short_description', 'T_0_feature_value', 
             'T_1_feature_value', 'T_2_feature_value', 'T_3_feature_value','T_4_feature_value']

# change col names and  reorder
df_bbva.columns = col_names
df_banorte.columns = col_names
df_santander = df_santander.reindex(columns=col_names)

#deleting unnesary parenthesis
columnas = banorte_df.columns
for columna in columnas:
      df_banorte[columna] = df_banorte[columna].str.replace("(","-") 
      df_banorte[columna] = df_banorte[columna].str.replace(")","")
      df_banorte[columna] = df_banorte[columna].str.replace(",","")
      df_bbva[columna] = df_bbva[columna].str.replace("(","-") 
      df_bbva[columna] = df_bbva[columna].str.replace(")","")
      df_bbva[columna] = df_bbva[columna].str.replace(",","")

# cleaning short descrption
df_bbva["short_description"] = df_bbva["short_description"].apply(lambda x: str_lower_rm_punctuation_rm_accents(str(x)))
df_banorte["short_description"] = df_banorte["short_description"].apply(lambda x: str_lower_rm_punctuation_rm_accents(str(x)))
df_santander["short_description"] = df_santander["short_description"].apply(lambda x: str_lower_rm_punctuation_rm_accents(str(x)))

# renaming important variables
df_bbva.iloc[2,3],  df_banorte.iloc[5,3],  df_santander.iloc[2,3]  = important_variables[0], important_variables[0], important_variables[0]
df_bbva.iloc[11,3], df_banorte.iloc[24,3], df_santander.iloc[20,3] = important_variables[1], important_variables[1], important_variables[1]
df_bbva.iloc[20,3], df_banorte.iloc[34,3], df_santander.iloc[29,3] = important_variables[2], important_variables[2], important_variables[2]
df_bbva.iloc[28,3], df_banorte.iloc[35,3], df_santander.iloc[30,3] = important_variables[3], important_variables[3], important_variables[3]
df_bbva.iloc[30,3], df_banorte.iloc[39,3], df_santander.iloc[32,3] = important_variables[4], important_variables[4], important_variables[4]
df_bbva.iloc[41,3], df_banorte.iloc[54,3], df_santander.iloc[41,3] = important_variables[5], important_variables[5], important_variables[5]
df_banorte['table_name'] = 'Balance General Activo'

result_active = pd.concat([df_bbva,df_banorte,df_santander])

#to float
result_active["T_0_feature_value"] = result_active["T_0_feature_value"].astype(float)
result_active["T_1_feature_value"] = result_active["T_1_feature_value"].astype(float)
result_active["T_2_feature_value"] = result_active["T_2_feature_value"].astype(float)
result_active["T_3_feature_value"] = result_active["T_3_feature_value"].astype(float)
result_active["T_4_feature_value"] = result_active["T_4_feature_value"].astype(float)

#new vars 
result_active["T_5_feature_value"] = 0
result_active["T_6_feature_value"] = 0



In [ ]:
#concat
result_total = pd.concat([result_state,result_pasive,result_active])

In [ ]:
result_total.loc[result_total["short_description"]=="margen financiero"]

,trimester_date,bank_name,table_name,short_description,T_0_feature_value,T_1_feature_value,T_2_feature_value,T_3_feature_value,T_4_feature_value,T_5_feature_value,T_6_feature_value
7,2021T2,BBVA,Estado de resultados,margen financiero,34409.0,34759.0,35914.0,34045.0,29089.0,0,0
8,2021T2,Banorte,Estado de resultados,margen financiero,19649.0,20002.0,19580.0,20601.0,20683.0,0,0
0,2021T2,Santander,Estado de resultados,margen financiero,15585.0,16272.0,16896.0,15585.0,16896.0,0,0


In [ ]:
result_total.loc[result_total["short_description"]=="total pasivo y capital contable"]

,trimester_date,bank_name,table_name,short_description,T_0_feature_value,T_1_feature_value,T_2_feature_value,T_3_feature_value,T_4_feature_value,T_5_feature_value,T_6_feature_value
48,2021T2,BBVA,Balance General Pasivos,total pasivo y capital contable,2708462.0,2709335.0,2704552.0,2688394.0,2779781.0,0,0
57,2021T2,Banorte,Balance General Pasivos,total pasivo y capital contable,1802363.0,1810207.0,1787904.0,1795306.0,1783581.0,0,0
40,2021T2,Santander,Balance General Pasivos,total pasivo y capital contable,1634384.0,1748298.0,1856213.0,1858684.0,1929350.0,0,0


In [ ]:
variables = ['Inversiones en Valores', 'Total Cartera de Credito Vigente', 'Total Cartera de Credito Vencida',
                       'Total Cartera de Credito', 'Total de Cartera de Credito (Neto)', 'Total Activo',
             "resultado neto","margen financiero","margen financiero ajustado por riesgos crediticios",
                                    "resultado de la operacion","resultado antes de impuestos a la utilidad",
                                    "resultado antes de operaciones discontinuadas","resultado neto",
             "prestamos interbancarios y de otros organismos",
                                    "colaterales vendidos o dados en garantia",
                                    "derivados",
                                    "otras cuentas por pagar",
                                    "total pasivo",
                                    "capital contribuido",
                                    "capital ganado",
                                    "total capital contable",
                                    "total pasivo y capital contable"]

In [ ]:
result_total.to_csv(path+"general_result_state.csv", index = False)

In [ ]:
archivo = "general_result_state.csv"

In [ ]:
client_s3 = boto3.client(
                         's3',
                         aws_access_key_id = access_key,
                         aws_secret_access_key = secret_key,
                        )

#for file in os.listdir(path):
try:
    print("Subiendo archivo: ", archivo)
    client_s3.upload_file(
                           path + archivo,
                           bucket_name,
                           "calamar_" + archivo
      
                         )
    print("Archivo subido con éxito.")
#except ClientError as e:
#     print("Invalid credentials.")
#     print(e)
except Exception as e:
     print("Error.")
     print(e)

Subiendo archivo:  general_result_state.csv
Archivo subido con éxito.
